In [ ]:
# Set Day Month Year
from datetime import datetime, timedelta

today = datetime.utcnow()
year = today.year
month = today.month
day = today.day

In [ ]:
# Recursive data load for all files from a day from every partition in the Event Hub Namespace
sourcefolderpath = f"abfss://crypto-data@dltalakehousesilver.dfs.core.windows.net/quotes-by-day/{year}/{month:0>2d}/{day:0>2d}"

df = spark.read.parquet(sourcefolderpath)

In [ ]:
# Add Price Date column
from pyspark.sql.functions import to_date

df = df.withColumn("PriceDate", to_date("PriceTimeStamp"))

In [ ]:
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")

In [ ]:
sql_table_create = "CREATE TABLE IF NOT EXISTS gold.crypto_fact \
                      (Symbol STRING NOT NULL, Price DECIMAL(38,15) NOT NULL, VolumeLastHourUSD DECIMAL(38,15) NOT NULL, PriceTimeStamp TIMESTAMP NOT NULL, PriceDate DATE NOT NULL) \
                    USING DELTA \
                    PARTITIONED BY (PriceDate) \
                    LOCATION 'abfss://crypto-data@dltalakehousegold.dfs.core.windows.net/crypto-fact'"

spark.sql(sql_table_create)